In [ ]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors, Word2Vec
import os
import re
import random
!pip install docx2txt
import docx2txt

  Created wheel for docx2txt: filename=docx2txt-0.8-cp36-none-any.whl size=3963 sha256=a8c7e08409f6e02893580553b0aa374e92a326c7a0e67542d024889fef98576d
  Stored in directory: /root/.cache/pip/wheels/b2/1f/26/a051209bbb77fc6bcfae2bb7e01fa0ff941b82292ab084d596
Successfully built docx2txt


In [ ]:
#Ubah direktori sesuai lokasi penyimpanan dataset
DIR_DATA_MHS = '/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/jawaban_mahasiswa/'
DIR_DATA_DSN = '/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/jawaban_dosen/'
DIR_W2V_MODELS = '/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/word2vec_models/'

In [ ]:
!pip install stop_words
from stop_words import get_stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=ed50b5aee56832787d9a4c5c07e1e32f11ed32d95d36f5fccff4068db89a2ed5
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [ ]:
def preprocess(text):
  text = re.sub("[^A-Za-z0-9]", " ", text)
  text = re.sub("^[0-9]", " ", text)
  words = text.lower().split()
  stop_words = get_stop_words('indonesian')
  words = [word for word in words if not word in stop_words]
  return words

In [ ]:
def get_data(dirf,fname):
  documents = []
  for i in range(1,5):
    name = dirf+fname+str(i)+".docx"
    text = docx2txt.process(name)
    text = text.splitlines()
    text = [x for x in text if x]
    documents.append(text)

  fullDocs = [documents[i][x] for i in range(len(documents)) for x in range(len(documents[i]))]

  finalText = []
  for doc in fullDocs:
    text = preprocess(doc)
    finalText.append(text)
  return finalText

finalText = get_data(DIR_DATA_DSN,'jwbdosen')
print(finalText)

[['von', 'neumann', 'komputer', 'pertama', 'menggunakan', 'konsep', 'penyimpanan', 'dimana', 'komponen', 'komponennya', 'prosesor', 'memori', 'input', 'output'], ['hibernate', 'salah', 'satu', 'proses', 'dimana', 'komputer', 'dimatikan', 'dimatikan', 'data', 'berupa', 'program', 'dijalankan', 'berada', 'ram', 'dialihkan', 'hard', 'disk', 'rom', 'data', 'hilang', 'jadi', 'dinyalakan', 'kembali', 'data', 'berupa', 'program', 'dinyalakan', 'dikembalikan', 'ram', 'rom', 'seakan', 'komputer', 'dimatikan', 'sepenuhnya'], ['server', 'host', 'menjadi', 'tempat', 'menyimpan', 'data', 'informasi', 'dituju', 'diakses', 'client', 'jaringan', 'client', 'host', 'menjadi', 'alat', 'untuk', 'mengakses', 'meminta', 'data', 'informasi', 'dikirim', 'server', 'client', 'berfungsi', 'untuk', 'mengirimkan', 'data', 'informasi', 'simpan', 'server'], ['turbo', 'boost', 'cara', 'untuk', 'meningkatkan', 'kecepatan', 'clock', 'komputer', 'menjadi', 'kecepatan', 'maksimal', 'pengolahan', 'data', 'cepat', 'kecepat

In [ ]:
w2v_model = 'idwiki_word2vec_300.model'
id_w2v = Word2Vec.load(DIR_W2V_MODELS+w2v_model)
id_w2v.wv.most_similar('komputer')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('mikroprosesor', 0.6062144041061401),
 ('komputasi', 0.6013833284378052),
 ('laptop', 0.6006243228912354),
 ('perangkat', 0.6002838611602783),
 ('aplikasi', 0.579538881778717),
 ('cpu', 0.5764787197113037),
 ('mainframe', 0.5717548131942749),
 ('nirkabel', 0.5690869688987732),
 ('server', 0.5684555768966675),
 ('prosesor', 0.5614745020866394)]

In [ ]:
id_w2v_set = set(id_w2v.wv.index2word)

In [ ]:
!pip install PySastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def replace_words(sentences, idx):
  dummy_text = sentences
  for i in idx:
    token = sentences[i]
    if token in id_w2v_set:
      similar_words = id_w2v.wv.most_similar(sentences[i], topn=10)
      random_index = random.randint(0,2)
      # similar_word = stemmer.stem(similar_words[random_index][0])  
      similar_word = similar_words[random_index][0]            
      dummy_text[i] = similar_word
  return dummy_text

def generate_dummy(sentences,score):
  size = len(sentences)    
  keywords = ['von', 'neumann', 'hibernate', 'server', 'client', 'host', 'turbo', 'boost', 'clock']
  indexes = [i for i in range(size) if sentences[i] not in keywords]
  num_replaced_words = round(((100-score)/100)*len(indexes))  
  
  if num_replaced_words == 0 or score == 100:
    return sentences
  elif num_replaced_words == len(indexes) or score == 0:
    idx = [i for i in range(size)]
    return replace_words(sentences, idx)  
  else:
    idx_to_replace = random.sample(indexes, num_replaced_words)    
    dummy_text = replace_words(sentences, idx_to_replace)
    return dummy_text 
    
  return None,-1

In [ ]:
print(finalText[0])

['von', 'neumann', 'komputer', 'pertama', 'menggunakan', 'konsep', 'penyimpanan', 'dimana', 'komponen', 'komponennya', 'prosesor', 'memori', 'input', 'output']


In [ ]:
txt = finalText[1]
print(txt)
print(generate_dummy(txt[:],90))

# print(finalText[1])
# print(generate_dummy(finalText[1][:],10))

# print(finalText[2])
# print(generate_dummy(finalText[2][:],10))

# print(finalText[3])
# print(generate_dummy(finalText[3][:],10))

['hibernate', 'salah', 'satu', 'proses', 'dimana', 'komputer', 'dimatikan', 'dimatikan', 'data', 'berupa', 'program', 'dijalankan', 'berada', 'ram', 'dialihkan', 'hard', 'disk', 'rom', 'data', 'hilang', 'jadi', 'dinyalakan', 'kembali', 'data', 'berupa', 'program', 'dinyalakan', 'dikembalikan', 'ram', 'rom', 'seakan', 'komputer', 'dimatikan', 'sepenuhnya']
['hibernate', 'satnya', 'satu', 'proses', 'dimana', 'komputer', 'dimatikan', 'dihilangkan', 'datanya', 'berupa', 'program', 'dijalankan', 'berada', 'ram', 'dialihkan', 'hard', 'disk', 'rom', 'data', 'hilang', 'jadi', 'dinyalakan', 'kembali', 'data', 'berupa', 'program', 'dinyalakan', 'dikembalikan', 'ram', 'rom', 'seakan', 'komputer', 'dimatikan', 'sepenuhnya']


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
scores = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 0]
dummy_corpus = []
ref_corpus = []
labels = []
copy = 10

In [ ]:
for i in range(copy):
  for score in scores:
    for j in range(len(finalText)):
      text = finalText[j]
      dummy = generate_dummy(text[:], score)
      dummy_text = ' '.join(str(t) for t in dummy)
      dummy_corpus.append(dummy_text)
      ref = ' '.join(text)
      ref_corpus.append(ref)
      labels.append(score)

print(len(dummy_corpus))
print(len(ref_corpus))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


440
440


In [ ]:
for i in range(20):print(dummy_corpus[i])

von neumann komputer pertama menggunakan konsep penyimpanan dimana komponen komponennya prosesor memori input output
hibernate salah satu proses dimana komputer dimatikan dimatikan data berupa program dijalankan berada ram dialihkan hard disk rom data hilang jadi dinyalakan kembali data berupa program dinyalakan dikembalikan ram rom seakan komputer dimatikan sepenuhnya
server host menjadi tempat menyimpan data informasi dituju diakses client jaringan client host menjadi alat untuk mengakses meminta data informasi dikirim server client berfungsi untuk mengirimkan data informasi simpan server
turbo boost cara untuk meningkatkan kecepatan clock komputer menjadi kecepatan maksimal pengolahan data cepat kecepatan dimaksud kecepatan pengolahan transmisi data komputer
von neumann komputer pertama menggunakan konsep penyimpanan dimana komponen komponennya cpu memori input output
hibernate salah satu proses dimana komputer dimatikan dimatikan data berupa paket dijalankan berada ram dialihkan ha

In [ ]:
for i in range(20):print(labels[i])

100
100
100
100
90
90
90
90
80
80
80
80
70
70
70
70
60
60
60
60


In [ ]:
print(len(labels))

440


In [ ]:
table = {'jawaban_dummy':dummy_corpus,'jawaban_dosen':ref_corpus,'nilai_dummy':labels}
df = pd.DataFrame(table)
print(df.head())

                                       jawaban_dummy  ... nilai_dummy
0  von neumann komputer pertama menggunakan konse...  ...         100
1  hibernate salah satu proses dimana komputer di...  ...         100
2  server host menjadi tempat menyimpan data info...  ...         100
3  turbo boost cara untuk meningkatkan kecepatan ...  ...         100
4  von neumann komputer pertama menggunakan konse...  ...          90

[5 rows x 3 columns]


In [ ]:
df.to_csv (r'/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/dummy_dataset3.csv', index = False, header=True)